<a href="https://colab.research.google.com/github/hoomanabasi/Text-Summarization-with-T5-Model/blob/main/Text_Summarization_with_T5_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [2]:
from transformers import T5ForConditionalGeneration,T5Tokenizer
from datasets import load_dataset

In [3]:
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")
model=T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer=T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
articel=ds['train'][0]['article']
input=tokenizer(articel,return_tensors="pt",max_length=2048,truncation=True,padding="max_length")
summery_ids=model.generate(input["input_ids"],max_length=150,min_length=40,length_penalty=5.,num_beams=2)
summery=tokenizer.decode(summery_ids[0],skip_special_tokens=True)
print(summery)
print(articel)

"I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection," he said . "the things I like buying are things that cost about 10 pounds," he said in an interview . despite his growing fame and wealth, the young actor says he is keeping his feet firmly on the ground . he made his stage debut playing a tortured teenager in "Equus" earlier this year .
LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or some

In [13]:
import evaluate

# بارگذاری متریک ROUGE
rouge = evaluate.load("rouge")

def compute_metrics(pred):
    predictions, labels = pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # پاک کردن -100 ها از لیبل‌ها
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # محاسبه متریک
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # فقط ROUGE1، ROUGE2 و ROUGEL را نگه می‌داریم
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
